# simpson's paradox quadratic programming

In a previous post, we discussed that when subgroup sizes are _exactly_ balanced in terms of number of data points, then Simpson's paradox becomes impossible.

One interesting question we could ask is: given a particular level of difference between the sample means $\epsilon$, how big of an imbalance in the group sizes do we need to attain that? 


In particular, let $t_i$ and $c_i$ be as above, but $g_{t0}, g_{c0}, g_{t1}, g_{c1}$ denote the group sizes of the four groups. Then we have the following constraints:


\begin{align}
 \tag{1} (1-\epsilon)\cdot\frac{t_0}{g_{t0}} &> \frac{c_0}{g_{c0}} \\
 \tag{2} (1-\epsilon)\cdot\frac{t_1}{g_{t1}} &> \frac{c_1}{g_{c1}} \\
 \tag{3} \frac{t_0 + t_1}{g_{t0} + g_{t1}} &< \frac{c_0 + c_1}{g_{c0} + g_{c1}} \\
 \tag{4} g_{t0} & \geq t_0 \\
 \tag{5} g_{c0} & \geq c_0 \\
 \tag{6} g_{t1} & \geq t_1 \\
 \tag{7} g_{c1} & \geq c_1 \\
\end{align}

This is a quadratic program, but we cannot use [optlang](https://pypi.org/project/optlang/) to solve it, since the constraints are quadratic rather than linear. For our objective function, we can assume that $g_{t0} = 1$, and then minimize:
$$ (g_{c0} - 1)^2 + (g_{t1} - 1)^2 + (g_{c1} - 1)^2 $$

Making this substitution, and cross-multiplying denominators to get a quadratic program, we get:
\begin{align}
 \tag{1} (1-\epsilon) \cdot t_0 \cdot g_{c0}  - c_0 &> 0 \\
 \tag{2} (1-\epsilon)\cdot t_1\cdot g_{c1} - c_1\cdot g_{t1} &> 0 \\
 \tag{3} -(t_0 + t_1)\cdot (g_{c0} + g_{c1}) + (c_0 + c_1) \cdot (1 + g_{t1}) &> 0\\
 \tag{4} 1 - t_0 & \geq 0 \\
 \tag{5} g_{c0} - c_0 & \geq 0 \\
 \tag{6} g_{t1} - t_1 & \geq 0 \\
 \tag{7} g_{c1} - c_1 & \geq 0 \\
\end{align}

In [1]:
from optlang import Model, Variable, Constraint, Objective

def nonnegative_vars(comma_separated_names: str):
    return [Variable(name.strip(), lb=0) for name in comma_separated_names.split(',')]

def solve_optimization(eps):
    t0, c0, t1, c1, g_c0, g_t1, g_c1 = nonnegative_vars('t0, c0, t1, c1, g_c0, g_t1, g_c1')
    eqn_num_to_constraint = {
        "(1)": Constraint((1 - eps) * t0 * g_c0 - c0, lb=0),
        "(2)": Constraint((1-eps) * t1 * g_c1 - c1 * g_t1, lb=0),
        "(3)": Constraint(-(t0 + t1) * (g_c0 + g_c1) + (c0 + c1) * (1 + g_t1), lb=0),
        "(4)": Constraint(1 - t0, lb=0),
        "(5)": Constraint(g_c0 - c0, lb=0),
        "(6)": Constraint(g_t1 - t1, lb=0),
        "(7)": Constraint(g_c1 - c1, lb=0),
    }
    objective = Objective((g_c0 - 1) ** 2 + (g_t1 - 1) ** 2 + (g_c1 - 1) ** 2, direction="min")
    model = Model(name=f"Simpson finder epsilon={eps}")
    model.add(list(eqn_num_to_constraint.values()))
    status = model.optimize()
    
solve_optimization(1/10)
    

ValueError: GLPK only supports linear constraints. ee72425a-f19c-11ea-bdcf-acde48001122: 0 <= -c0 + 0.9*g_c0*t0 is not linear.

## TODO: Use NonlinearConstraint in scipy.optimize, with sympy computing the Jacobian and Hessian.